In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch 
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt

In [3]:
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer

        #to store activations and gradients
        self.activation = None
        self.gradients = None

        ##HOOK THE FORWARD AND BACKWARD PASSES
        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_backward_hook(self.save_gradients)

    def save_activation(self, module, input, output):
        self.activations = output.detach()

    def save_gradients(slf, module, grad_input, grad_output):
        self.gradients = grad_output.detach()

    def generate(self, input_tensor, class_idx = None):
        """
        input_tensor: single image tensor [1, C, H, W]
        class_idx: class index for which Grad-CAM is computed (default = predicted class)
        """

        #forward pass
        output = self.model(input_tensor)

        if class_idx is None:
            class_idx = output.argmax(dim=1).items()

        ##backward pass: get gradients wrt chosen class
        self.model.zero_grad()
        